In [ ]:
section_ind = 3
locs = [np.load('/home/yuncong/csd395/top3LocsLm%dSect%d.npy'%(lm_ind, section_ind)) for lm_ind in range(6)]

In [ ]:
centroids = []
for lm_ind in range(6):
    segment_points = np.load('/home/yuncong/csd395/sample_landmark_%d.npy'%lm_ind)
    centroid = np.mean(segment_points[:, [1,0]], axis=0)
    centroids.append(centroid)

centroids = np.array(centroids)

In [ ]:
def rigid_transform_from_pairs(X,Y):
    '''
    X, Y are n-by-2 matrices
    Y = RX
    '''
    
    Xcentroid = X.mean(axis=0)
    Ycentroid = Y.mean(axis=0)
    
    Xcentered = X - Xcentroid
    Ycentered = Y - Ycentroid
    
    U, S, V = np.linalg.svd(np.dot(Xcentered.T, Ycentered))
    M = [[1, 0], [0, np.linalg.det(np.dot(V, U.T))]]
    R = np.dot(V, np.dot(M, U.T))
    angle = np.arctan2(R[1,0], R[0,0])
    t = Ycentroid.T - np.dot(R, Xcentroid.T)
    
    T = np.eye(3)
    T[:2, :2] = R
    T[:2, 2] = t
   
    return T, angle

def rigid_transform_to(pts1, T):
    pts1_trans = np.dot(T, np.column_stack([pts1, np.ones((pts1.shape[0],))]).T).T
    pts1_trans = pts1_trans[:,:2]/pts1_trans[:,-1][:,np.newaxis]
    return pts1_trans


In [ ]:
n_inliers = []
rewards = []
Ts = []
As = []
all_matches = []
for ransac_i in range(100):

    lm_samples = np.random.randint(0,6,3)
    samples = np.array([np.r_[locs[l][np.random.randint(0,3), :2], centroids[l]] for l in lm_samples]).astype(np.int)
    T, a = rigid_transform_from_pairs(samples[:,:2], samples[:,2:])

    n_inlier = 0
    reward = 0
    pairs = []
    matches = []
    for l in range(6):
        vecs = rigid_transform_to(locs[l][:,:2], T) - centroids[l]
        ds = np.sqrt(np.sum(vecs**2, axis=1))
        nn = ds.argmin()
        if ds[nn] < 300:
            n_inlier += 1
            reward += np.exp(-ds[nn]/400.)  # !!! use response map to evaluate goodness
            matches.append((l,nn))
            pairs.append(np.r_[locs[l][nn,:2], centroids[l]])
            
            
    n_inliers.append(n_inlier)
    rewards.append(reward)
    
    if len(pairs) > 0:            
        pairs = np.array(pairs)
        T, a = rigid_transform_from_pairs(pairs[:,:2], pairs[:,2:])
        Ts.append(T)
        As.append(a)
        all_matches.append(matches)
    else:
        Ts.append(None)
        As.append(None)
        all_matches.append(None)
        
best_iter = np.argmax(rewards)

print n_inliers[best_iter]
print rewards[best_iter]
print all_matches[best_iter]

In [ ]:
np.abs(As[best_iter] - thetas).argmin()

In [ ]:
colors = 'rgbkcy'
for l in range(6):
    plt.scatter(locs[l][:,1], locs[l][:,0], c=colors[l], s=50);
    plt.scatter(centroids[l, 1], centroids[l, 0], marker='^', c=colors[l], s=50);
    
plt.show()

In [ ]:
colors = 'rgbkcy'
for l in range(6):
    tr = rigid_transform_to(locs[l][:,:2], Ts[best_iter])
    plt.scatter(tr[:,1], tr[:,0], c=colors[l], s=50);
    plt.scatter(centroids[l, 1], centroids[l, 0], marker='^', c=colors[l], s=50);
    
plt.show()

In [ ]:
from skimage.feature import peak_local_max

peaks = peak_local_max(vs_max_smooth)
ypeaks = peaks[:,0]
xpeaks = peaks[:,1]

order = np.argsort(vs_max_smooth[ypeaks, xpeaks])[::-1]
ypeaks = ypeaks[order]
xpeaks = xpeaks[order]


bg_ubyte = img_as_ubyte(dm.image_rgb)

for i, (ybest, xbest) in enumerate(zip(ypeaks, xpeaks)[:10]):

    best_theta = vs_argmax[ybest, xbest]
    
    print ybest, xbest, vs_max[ybest, xbest], best_theta

    shifted_ys_best = ybest + template_points_rotated_versions[best_theta][:,0]
    shifted_xs_best = xbest + template_points_rotated_versions[best_theta][:,1]
    
    overlay = np.zeros_like(dm.image, np.bool)
    overlay[shifted_ys_best, shifted_xs_best] = 1
    overlay = binary_dilation(overlay, disk(5))
    bg_ubyte[overlay] = [0,255,0]

    cv2.putText(bg_ubyte, str(i), (xbest, ybest), 
                cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 3, ((255,0,255)), 10)

display(bg_ubyte)